In [15]:
% matplotlib inline
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns
import nltk
import re
from ipywidgets import interact, interactive, fixed, HBox, widgets
from collections import Counter
import operator

In [16]:
otc = pd.read_csv("C:\\Users\\Nikhil\\Documents\\DrugDatabase\\fake data.csv")
otc.head()

,Drug,Indication,Condition,Effectiveness,EaseofUse,Satisfaction,Reviews,Form,PkgCount,EstimatedPrice,Allergies
0,Deconex DMX oral,label,Cough,4.400000,4.600000,4.000000,5.0,tablet,30.0,26.9236,Acacia
1,Duraflu oral,label,Cough,4.666667,5.000000,4.500000,6.0,tablet,20.0,26.3628,Acacia
2,chlorpheniramine oral,offlabel,Cough,4.107143,4.678571,4.035714,28.0,tablet,60.0,21.2601,Acacia
3,Chlor-Trimeton oral,offlabel,Cough,4.625000,4.687500,4.500000,16.0,tablet,60.0,21.2601,Acacia
4,Deconex IR oral,label,Cough,3.857143,4.285714,3.714286,7.0,tablet,20.0,19.9769,Acacia


In [17]:
#['Use','HowtoUse','Precautions','AvoidUse','Allergies','Interactions','Sides''BrandName','GenName',] # columns removed
def search_conditions(string, tg2, tg3, age, dd1, tg1, na, conds, form, count, revrange, currmed, health):
    search = ['Indication','Condition','Drug',
              'Effectiveness','EaseofUse','Satisfaction',
              'Reviews','Form','PkgCount','EstimatedPrice']
    index = []
    a1index = []
    df = otc.copy()
    if na:
        df = df.loc[(df.EaseofUse >= 0)&(df.EstimatedPrice >= 0)].reset_index(drop=True)
    
    if revrange:    
        df = df.loc[(df.Reviews >= revrange)].reset_index(drop=True)
    
    if form != 'All':
        df = df.loc[df.Form == form].reset_index(drop=True)
        
    if conds != '':
        conds = '(' + ')|('.join(conds) + ')'
    for i in range(len(df)):
        if re.search(str(conds).lower(), 
                     df.Allergies[i].lower()
                    ):
            a1index += [i]
        else:
            continue
    df = df[~df.index.isin(a1index)].reset_index(drop=True)
                    
    if tg2:
        for i in range(len(df)):
            if re.search('^'+string.lower()+'$', df.Condition[i].lower()):
                index += [i]
            else:
                continue
        return df[search].loc[index].sort_values(dd1, ascending=tg1).head(count).set_index('Drug')
    
    if tg3:
        regex = '^(?=.*\\b'+'\\b)(?=.*\\b'.join(string.lower().split())+'\\b).*$'
        for i in range(len(df)):
            if re.search(regex, df.Condition[i].lower()):
                index += [i]
            else:
                continue
        return df[search].loc[index].sort_values(dd1, ascending=tg1).head(count).set_index('Drug')
    
    else:
        for i in range(len(df)):
            if re.search(string.lower(), df.Condition[i].lower()):
                index += [i]
            else:
                continue
        return df[search].loc[index].sort_values(dd1, ascending=tg1).head(count).set_index('Drug') 
    
# search_conditions('cough',False,'False','','Effectiveness',False,True,'All',5) # for debugging only

### Widget Creation

In [18]:
allergstr = ['Acacia','Acetic Acid','Acerola']
#reg = re.compile(r"[\d+\w+]{2,}")
#for i in range(len(otc.Allergies)):
#    allergstr += nltk.word_tokenize(otc.Allergies[i])
#allergstr = [x for x in list(set(allergstr)) if reg.match(x)]

In [19]:
tg1 = widgets.ToggleButton(
    value=False,
    description='Ascending/Descending',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

tg2 = widgets.ToggleButton(
    value=False,
    description='Match Exact',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

tg3 = widgets.ToggleButton(
    value=False,
    description='Match Term',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

dd1 = widgets.Dropdown(
    options={'Estimated Price': 'EstimatedPrice', 'Reviews': 'Reviews', 'Ease of Use': 'EaseofUse',\
            'Satisfaction': 'Satisfaction', 'Effectiveness': 'Effectiveness'},
    value='Reviews',
    description='Sort by:',
    disabled=False,
    button_style='info' # 'success', 'info', 'warning', 'danger' or ''
)

str1 = widgets.Text(
    value='Cough',
    placeholder='',
    description='Search:',
    disabled=False
)

int1 = widgets.IntSlider(
    value = 5,
    step = 1,
    min = 1,
    max = 10,
    description = 'Show:'
)

age1 = widgets.IntText(
    value = 35,
    step = 1,
    min = 1,
    max = 100,
    description = 'Age:'
)

na1 = widgets.ToggleButton(
    value=True,
    description='Remove NA',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

for1 = widgets.Dropdown(
    options=['All'] + otc.Form.unique().tolist(),
    value='All',
    description='Form:',
    disabled=False,
    button_style='info' # 'success', 'info', 'warning', 'danger' or ''
)

revrange = widgets.IntSlider(
    value= 5,
    min=0,
    max= 50,
    step=5,
    description='Minimum Reviews:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='i',
    slider_color='white',
    color='black'
)

a= widgets.SelectMultiple(
    options=['NONE'] +sorted(allergstr),
    value=['NONE'],
    description='Allergies:',
    disabled=False
)

currmed = widgets.Text(
    value='',
    placeholder='',
    description='Current Meds:',
    disabled=False
)

health = widgets.Text(
    value='',
    placeholder='',
    description='Known Illnesses:',
    disabled=False
)

### Drug Finder

In [20]:
interactive(search_conditions,
            string = str1,
            tg1 = tg1,
            tg3 = tg3,
            conds = a,
            age = age1,
            dd1 = dd1,
            na = na1,
            tg2 = tg2,
            form = for1,
            count = int1,
            revrange = revrange,
            currmed = currmed,
            health = health,
            __manual=False)

,Indication,Condition,Effectiveness,EaseofUse,Satisfaction,Reviews,Form,PkgCount,EstimatedPrice
Drug,,,,,,,,,
fake drug 5,label,Cold Symptoms,4.0,4.5,3.784615,1.0,liquid,15.0,16.482972
